In [1]:
#from casstools.notebook_tools import NotebookFile
from casstools import code_tools as ct
from casstools.path_parser import PathParser

In [2]:
import nbformat 

class NotebookFile(object):

    def __init__(self, filespec):
        self.filepec = filespec
        self.contents = self.load_notebook(filespec)

    def load_notebook(self, path: str):
        with open(path, "r") as f:
            return nbformat.read(f, as_version=4)

    def _extract_metadata_value(self, cell, key):
        '''
        Extracts metadata value from cell using key 
        first checks under ['metadata']['cass'][key]
        then under legaccy ['metadata'][key]
        '''
        md = cell['metadata']
        if md.get("cass", None) is not None:
            return md['cass'].get(key, None)
        else:
            return md.get(key, None)


    @property
    def code_cells(self):
        return [c for c in self.contents.cells if c.cell_type == "code"]

    @property
    def markdown_cells(self):
        return [c for c in self.contents.cells if c.cell_type == "markdown"]

    @property
    def raw_cells(self):
        return [c for c in self.contents.cells if c.cell_type == "raw"]

    def code_cells_of_type(self, cell_type, executed_only=False):
        '''
        return a list of code cells with the "cell_type" as their designation
        designation is found in:
        1) ['metadata']['cass']['code_cell_type'] or 
        2) ['metadata']['code_cell_type'] if the "cass" key does not exist.
        '''
        cells = [c for c in self.code_cells if self._extract_metadata_value(c, "code_cell_type") == cell_type]
        if executed_only:
            return [c for c in cells if c.execution_count is not None and c.execution_count >=1]
        else:
            return cells

    @property
    def exercise_code_cells(self):
        cells = [c for c in self.code_cells if self._extract_metadata_value(c, "code_cell_type") in ["debug_code", "write_code"]]
        return cells



In [3]:
# nb = NotebookFile(PathParser().fullpath)
# nb.code_cells

In [16]:
nb = NotebookFile("/home/jovyan/library/ist256/fall2023/lessons/02-Variables/LAB-Variables.ipynb")
total_run_code_cells = len(nb.code_cells_of_type("run_code"))
total_executed_run_code_cells = len(nb.code_cells_of_type("run_code", executed_only=True))

In [17]:
for cell in nb.exercise_code_cells:
    label = nb._extract_metadata_value(cell, "label")
    student_code = "\n".join([line for line in cell.source.split("\n") if not line.strip().startswith("#")])
    solution_code = "".join(nb._extract_metadata_value(cell, "solution"))
    syntax = ct.syntax_check(student_code)
    similarity = ct.code_similarity_check(solution_code, student_code)
    print(label, syntax, similarity)
    print(student_code, solution_code)

1.1 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 14, 'total_tokens': 14, 'pct_similar': 1.0}
age = int(input("Enter your age: "))
nextage = age + 1
print(f"Today you are {age} next year you will be {nextage}") age = int(input("Enter your age: "))
nextage = age + 1
print(f"Today you are {age} next year you will be {nextage}")

1.2 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 0, 'total_tokens': 8, 'pct_similar': 0.0}
 age = int(input("Enter your age: "))
nextage = age + 1
print("Today you are %d next year you will be %d" % (age,nextage))

1.3 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 0, 'total_tokens': 6, 'pct_similar': 0.0}
 PI = 3.1415927
print("%s" % PI)
print("%d" % PI)
print("%.4f" % PI)

1.4 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 0, 'total_tokens': 24, 'pct_similar': 0.0}
 length = float(input("Enter length of yard: "))
width = float(input("Enter width of yard: "))
perimeter = 2*(length + width)
se

In [6]:
syntax

{'ok': True, 'output': '', 'error': ''}

In [1]:
from assignment import Assignment
Assignment()._confirm_submission(False,True)

ℹ Submit Again? [y/n] ❓  ds


False

In [3]:
from datetime import datetime

In [17]:
assignment_due_date = "2024-09-20 23:59"
due = datetime.strptime(assignment_due_date, "%Y-%m-%d %H:%M")
now = datetime.now()

In [15]:
now.strftime("%Y-%m-%d %H:%M")

'2024-01-06 12:34'

In [21]:
now <= due

True

In [2]:
import pandas as pd

In [12]:
pd.read_json("data/demo.json", lines=True, orient="values")

,name,age
0,abby,10
1,bob,11
2,chris,12


In [5]:
from assignment import Assignment
Assignment().read_submission_log()

,TIMESTAMP,COURSE,TERM,USER,STUDENT,PATH,ASSIGNMENT,POINTS,DUE DATE,LATE,STATUS,RECIEPT
0,2024-01-06 13:14:00,ist256,spring2024,mafudge@syr.edu,True,ist256/spring2024/00-testing/testing.ipynb,testing.ipynb,5,2024-09-20 23:59,False,On Time,28158a2cb4b3c23fc46b7a169b287c3c
1,2024-01-06 13:15:00,ist256,spring2024,mafudge@syr.edu,True,ist256/spring2024/00-testing/testing.ipynb,testing.ipynb,5,2024-09-20 23:59,False,On Time,28158a2cb4b3c23fc46b7a169b287c3c
